In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import re
from datetime import datetime, timedelta
import requests
from time import sleep
import os
from zipfile import ZipFile 
from pandas import ExcelWriter

# Tablas
path_tablas = "G:/Petrini Valores/6. Reuters/"

usuario_mav = "petrini12"
contra_mav = "fernanda15"


n = int(input("Cuantos dias hay que tomar: ")) - 1

driver = webdriver.Chrome(executable_path=r"G:\Petrini Valores\8. Bloomberg\Informes\PyME\chromedriver_win32\chromedriver.exe")

url = "https://trading.mav-sa.com.ar/"

driver.get(url)

usuario = driver.find_element_by_xpath('//*[@id="UserLogin_username"]')
usuario.send_keys(usuario_mav)

contra = driver.find_element_by_xpath('//*[@id="UserLogin_password"]')
contra.send_keys(contra_mav)

driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div/form/p[3]/input').click()

driver.get('https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService%3Dwsbroker1/cpd-ch-operados.r')



Cuantos dias hay que tomar: 1


In [7]:
i = 1

while n >= 0:
    opciones = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr/td[2]/select')
    option = opciones.find_elements_by_tag_name('option')
    
    print(option[i].text)
    option[i].click()
    
    driver.find_element_by_xpath('/html/body/table[2]/tbody/tr/td[3]/a[2]/b').click()
    
    n -= 1
    i += 1

sleep(3*(i))

path = r"C:\Users\Bloomberg\Descargas"

xlsxfiles = [os.path.join(root, name) for root, dirs, files in os.walk(path) for name in files if name.endswith(".xls") and name.startswith("CPD-cheques-operados (")]


xlsxfiles.sort()
x = [xlsxfiles[-1]]

x.extend(xlsxfiles[:-1])

05/04/21


IndexError: list index out of range

In [44]:
driver.get('https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService=wsbroker1/logout.html')
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div/form/center/strong/strong/center/p[2]/input').click()

In [ ]:


driver.get('https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService=wsbroker1/logout.html')
driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div/form/center/strong/strong/center/p[2]/input').click()

tablas = pd.DataFrame({})

for excel in x:
    
    book = pd.read_html(excel)[0]
    
    book.columns = book.iloc[book.iloc[:,0].tolist().index("ID Cheque"),:].tolist()
    book = book.iloc[book.iloc[:,0].tolist().index("ID Cheque")+1:,:]
    
    tablas = pd.concat([tablas, book])
    os.remove(excel)
    print(excel + " Removed!")

tablas[["Plazo","Monto Nominal","Monto Liquidado"]] = tablas[["Plazo","Monto Nominal","Monto Liquidado"]].astype(float)
tablas.to_excel(path_tablas + "CDP-cheques-operados-" + datetime.today().strftime("%Y-%m-%d") + ".xlsx")

